In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision import datasets

import matplotlib.pyplot as plt
import os
from PIL import Image
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
path='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Pytorch/Convolutional neural networks/Vehicles'

In [3]:
train_transform=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

test_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

val_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),  
])

In [4]:
train_data=datasets.ImageFolder(os.path.join(path,'train'),transform=train_transform)
test_data=datasets.ImageFolder(os.path.join(path,'test'),transform=test_transform)  
val_data=datasets.ImageFolder(os.path.join(path,'val'),transform=val_transform)

In [5]:
print(f'Train data: {len(train_data)}')
print(f'Test data: {len(test_data)}')   
print(f'Validation data: {len(val_data)}')

Train data: 3542
Test data: 1777
Validation data: 583


In [6]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    model.train()
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    model.eval() 
    test_loss, test_acc = 0, 0
    
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            test_pred_logits = model(X)
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

from tqdm.auto import tqdm

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int):
    
    results={'train_loss':[],
             'train_acc':[],
             'test_loss':[],
             'test_acc':[]}
    
    for i in tqdm(range(epochs)):
        train_loss,train_acc=train_step(model=model,
                                        dataloader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer)
        test_loss,test_acc=test_step(model=model,
                                     dataloader=test_dataloader,
                                     loss_fn=loss_fn)
        print(f'Epoch {i} | Train loss: {train_loss:.4f},Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f},Test acc: {test_acc:.4f}')
        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

    return results

In [7]:
torch.manual_seed(101)
train_dl=DataLoader(train_data,batch_size=32,shuffle=True)
test_dl=DataLoader(test_data,batch_size=32,shuffle=False)

In [8]:
class_names=train_data.classes
class_names

['airplane', 'cruise ship', 'helicopter', 'motorbike', 'truck']

In [9]:
test_data.classes

['airplane', 'cruise ship', 'helicopter', 'motorbike', 'truck']

In [10]:
test_data.classes==train_data.classes

True

In [34]:
class Model1(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv=nn.Sequential(
            nn.Conv2d(3,32,3,1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.4),

            nn.Conv2d(32,64,3,1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64,128,3,1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.4),

            nn.Conv2d(128,128,3,1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Flatten()
        )

        self.fc=nn.Sequential(
            nn.Linear(25088,512),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(512,128),
            nn.ReLU(),

            nn.Linear(128,len(class_names))
        )

    def forward(self,X):
        return F.log_softmax(self.fc(self.conv(X)),dim=1)

In [35]:
torch.manual_seed(101)
model1=Model1()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model1.parameters(),lr=0.001)

In [36]:
dd=Model1()
x=torch.ones((32,3,224,224))
dd(x).shape

torch.Size([32, 5])

In [37]:
torch.manual_seed(42)

n=10
model_results=train(
    model=model1,
    train_dataloader=train_dl,
    test_dataloader=test_dl,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=n
)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 | Train loss: 1.4050,Train acc: 0.4209 | Test loss: 1.2686,Test acc: 0.5262
Epoch 1 | Train loss: 1.0202,Train acc: 0.5945 | Test loss: 1.1535,Test acc: 0.5344
Epoch 2 | Train loss: 0.8497,Train acc: 0.6689 | Test loss: 1.1348,Test acc: 0.5674
Epoch 3 | Train loss: 0.7734,Train acc: 0.7026 | Test loss: 0.7191,Test acc: 0.7184
Epoch 4 | Train loss: 0.7133,Train acc: 0.7201 | Test loss: 0.7819,Test acc: 0.7034
Epoch 5 | Train loss: 0.6464,Train acc: 0.7547 | Test loss: 0.6814,Test acc: 0.7334
Epoch 6 | Train loss: 0.5934,Train acc: 0.7753 | Test loss: 0.7381,Test acc: 0.7257
Epoch 7 | Train loss: 0.5659,Train acc: 0.7930 | Test loss: 0.5756,Test acc: 0.7760
Epoch 8 | Train loss: 0.4980,Train acc: 0.8075 | Test loss: 0.5241,Test acc: 0.8036
Epoch 9 | Train loss: 0.4515,Train acc: 0.8287 | Test loss: 0.4926,Test acc: 0.8254


In [39]:
index=list(range(1777))

c=0
for i in index:
    model1.eval()
    with torch.no_grad():
        pred=model1(test_data[i][0].view(1,3,224,224)).argmax()

    
    if class_names[test_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(test_data)} | Accuracy: {c*100/len(test_data)}')

Number of correct: 1473/1777 | Accuracy: 82.89251547552054


In [40]:
index=list(range(583))

c=0
for i in index:
    model1.eval()
    with torch.no_grad():
        pred=model1(val_data[i][0].view(1,3,224,224)).argmax()

    
    if class_names[val_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(val_data)} | Accuracy: {c*100/len(val_data)}')

Number of correct: 491/583 | Accuracy: 84.21955403087479


In [11]:
from torchvision import models

In [13]:
model2=models.resnet34(pretrained=True)
model2.fc=None
model2.fc=nn.Linear(512,len(class_names))

In [15]:
optimizer=torch.optim.Adam(model2.parameters(),lr=0.001)
criterion=nn.CrossEntropyLoss()

In [16]:
torch.manual_seed(42)

n=3
model2_results=train(
    model=model2,
    train_dataloader=train_dl,
    test_dataloader=test_dl,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=n
)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 0 | Train loss: 0.5032,Train acc: 0.8348 | Test loss: 0.5364,Test acc: 0.8019
Epoch 1 | Train loss: 0.3025,Train acc: 0.9011 | Test loss: 0.2963,Test acc: 0.9038
Epoch 2 | Train loss: 0.2048,Train acc: 0.9311 | Test loss: 0.3089,Test acc: 0.8991


In [19]:
index=list(range(1777))

c=0
for i in index:
    model2.eval()
    with torch.no_grad():
        pred=model2(test_data[i][0].view(1,3,224,224)).argmax()

    
    if class_names[test_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(test_data)} | Accuracy: {c*100/len(test_data)}')

Number of correct: 1597/1777 | Accuracy: 89.87056837366347


In [20]:
index=list(range(583))

c=0
for i in index:
    model2.eval()
    with torch.no_grad():
        pred=model2(val_data[i][0].view(1,3,224,224)).argmax()

    
    if class_names[val_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(val_data)} | Accuracy: {c*100/len(val_data)}')

Number of correct: 512/583 | Accuracy: 87.82161234991423
